In [1]:
using Random

module O

export TaskLocalLCG, LCG

using Random: Random, AbstractRNG, RandomDevice

const a = Int32(48271)
const m = Int32(2^31 - 1)
const q = m ÷ a
const r = m % a

struct TaskLocalLCG <: AbstractRNG end
const LCG = TaskLocalLCG()

@inline getstate(::TaskLocalLCG) = mod(current_task().rngState0, Int32)
@inline setstate!(lcg::TaskLocalLCG, x::Integer) = (current_task().rngState0 = x; lcg)

@inline function Random.rand(lcg::TaskLocalLCG) # 手抜き
    x = getstate(lcg)
    hi, lo = divrem(x, q)
    x = a * lo - r * hi
    x = ifelse(x > zero(x), x, x + m)
    setstate!(lcg, x)
    x / m # 本当は (x - 1) / m とするべき。
end

Random.seed!(lcg::TaskLocalLCG) = setstate!(lcg, rand(RandomDevice(), UInt64))
Random.seed!(lcg::TaskLocalLCG, x) = setstate!(lcg, x)

end

using .O

In [2]:
function mcpi_LCG(num_points = 10^9, seed = 20231226)
    Random.seed!(LCG, seed)
    num_inside = 0
    for i in 1:num_points
        num_inside += rand(LCG)^2 + rand(LCG)^2 < 1
    end
    4num_inside / num_points
end

@time mcpi_LCG()
@time mcpi_LCG()
@time mcpi_LCG()
@time mcpi_LCG()
@time mcpi_LCG()
@time mcpi_LCG()

  8.279241 seconds
  8.273855 seconds
  8.303930 seconds
  8.310941 seconds
  8.276474 seconds
  8.286418 seconds


3.141546348

In [3]:
using LoopVectorization

@inline isinside(i) = rand(LCG)^2 + rand(LCG)^2 < 1

function mcpi_LCG_turbo(num_points = 10^9, seed = 20231226)
    Random.seed!(LCG, seed)
    num_inside = 0
    @turbo for i in 1:num_points
        num_inside += isinside(i)
    end
    4num_inside / num_points
end

@time mcpi_LCG_turbo()
@time mcpi_LCG_turbo()
@time mcpi_LCG_turbo()
@time mcpi_LCG_turbo()
@time mcpi_LCG_turbo()
@time mcpi_LCG_turbo()

  1.032586 seconds
  1.037734 seconds
  1.028580 seconds
  1.037035 seconds
  1.030685 seconds
  1.031767 seconds


3.141605344

In [4]:
function mcpi_LCG_tturbo(num_points = 10^9, seed = 20231226)
    Random.seed!(LCG, seed)
    num_inside = 0
    @tturbo for i in 1:num_points
        num_inside += isinside(i)
    end
    4num_inside / num_points
end

mcpi_LCG_tturbo()
@time mcpi_LCG_tturbo()
@time mcpi_LCG_tturbo()
@time mcpi_LCG_tturbo()
@time mcpi_LCG_tturbo()
@time mcpi_LCG_tturbo()
@time mcpi_LCG_tturbo()

  0.199430 seconds (2 allocations: 32 bytes)
  0.211352 seconds (4 allocations: 192 bytes)
  0.208771 seconds (123 allocations: 10.609 KiB)
  0.227069 seconds (4 allocations: 216 bytes)
  0.192410 seconds
  0.212083 seconds (4 allocations: 152 bytes)


3.141618944